<a href="https://colab.research.google.com/github/neild0/StyleDrop-PyTorch-Interactive/blob/main/styledrop_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleDrop Demo

<p align="left">
  <a href="https://huggingface.co/spaces/zideliu/styledrop"><img alt="Huggingface" src="https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-StyleDrop-orange"></a>
  <a href="https://replicate.com/cjwbw/styledrop"><img src="https://replicate.com/cjwbw/styledrop/badge"></a>
</p>

This is an unofficial PyTorch implementation of [StyleDrop: Text-to-Image Generation in Any Style](https://arxiv.org/abs/2306.00983), based on [zideliu's implementation](https://github.com/zideliu/StyleDrop-PyTorch).

<img src="https://raw.githubusercontent.com/zideliu/StyleDrop-PyTorch/main/img/1.png"/>
<img src="https://raw.githubusercontent.com/zideliu/StyleDrop-PyTorch/main/img/2.png"/>
<img src="https://raw.githubusercontent.com/zideliu/StyleDrop-PyTorch/main/img/5.png"/>


**You need Colab Pro to train this model, with A100 enabled (~20Gb GPU RAM).**

In [1]:
!nvidia-smi

Sun Jul  9 07:22:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setup

In [ ]:
!git clone https://github.com/neild0/StyleDrop-PyTorch-Interactive
!mv StyleDrop-PyTorch-Interactive StyleDrop-PyTorch

!pip install omegaconf gdown accelerate==0.12.0 absl-py ml_collections einops wandb ftfy==6.1.1 transformers==4.23.1 loguru webdataset==0.2.5 gradio xformers

In [ ]:
!git lfs install
!git clone https://huggingface.co/nzl-thu/MUSE
!mv MUSE/assets/* StyleDrop-PyTorch/assets/
!rm -rf MUSE

In [ ]:
!gdown --id '13S_unB87n6KKuuMdyMnyExW0G1kplTbP' --output StyleDrop-PyTorch/assets/vqgan_jax_strongaug.ckpt

In [ ]:
!python StyleDrop-PyTorch/extract_empty_feature.py

#Set Style

In [ ]:
#@title Upload Style File

import shutil
from google.colab import files
import os
from IPython.display import Image, display

# specify the directory you want to move the file to
directory = "/content/StyleDrop-PyTorch/data"

# get all the files in the directory
uploaded = files.upload()

if len(uploaded.keys()) != 1:
    raise Exception("Please only select one file")

for fn in uploaded.keys():

    extension = os.path.splitext(fn)[1]

    if extension == '.jpg':
        # move the file and get the new path
        new_path = shutil.move(fn, f"{directory}/{fn.replace(' ', '_')}")

        # display the image
        display(Image(filename=new_path))

    else:
        raise Exception("Please only select jpg files")


In [12]:
#@title Describe style image
import json

object_in_image = "A biblical scene with a kneeling apostle, soldiers, and a king on a throne surrounded by attendants" #@param {type:"string"}
style_of_image = "early Renaissance painting" #@param {type:"string"}

file_name = os.path.basename(new_path)
one_style = {file_name: [object_in_image, f"in {style_of_image} style"]}
with open("/content/StyleDrop-PyTorch/data/one_style.json", "w") as f:
    json.dump(one_style, f)

# Train model

In [15]:
import os
import subprocess
import glob

if 'EVAL_CKPT' in os.environ:
    del os.environ['EVAL_CKPT']

if 'ADAPTER' in os.environ:
    del os.environ['ADAPTER']

style_output_folder = style_of_image.replace(" ", "_")

os.environ['OUTPUT_DIR'] = style_output_folder

!cd StyleDrop-PyTorch && accelerate launch --mixed_precision fp16 train_t2i_colab_v2.py --config=configs/custom.py

# get a list of all subdirectories
subdirs = glob.glob(f"/content/StyleDrop-PyTorch/{style_output_folder}/ckpts_II/" + '*/')
subdirs.sort()

last_subdir = subdirs[-1]
adapter = f"{last_subdir}/adapter.pth"

new_adapter_path = f"/content/StyleDrop-PyTorch/style_adapter/{style_output_folder}"
os.makedirs(new_adapter_path, exist_ok=True)
os.rename(adapter, f"{new_adapter_path}/adapter.pth")


# Run Gradio Interface

In [ ]:
!cd StyleDrop-PyTorch && python3 gradio_demo.py